# Medallion Architecture

**Cel szkoleniowy:** Opanowanie implementacji architektury medalionowej (Bronze → Silver → Gold) w Databricks.

**Zakres tematyczny:**
- Bronze Layer: Ingestion surowych danych (CSV, JSON, Parquet)
- Silver Layer: Data Quality, Deduplication & Validation, Kalkulacje biznesowe
- SCD (Slowly Changing Dimensions): Implementacja SCD Type 1 i Type 2
- Gold Layer: Star Schema (Fact & Dimension Tables)
- Best Practices: Partycjonowanie, Z-Ordering, Data Retention

**Model danych zgodny z Lakeflow Pipeline:**
- `bronze_customers`, `bronze_orders`, `bronze_products`
- `silver_customers` (SCD2), `silver_orders`, `silver_products`
- `fact_sales`, `dim_customer`, `dim_product`, `dim_date`, `dim_store`, `dim_payment_method`

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 2 - Delta Lake & Lakehouse
- **Typ notebooka**: Demo
- **Wymagania techniczne**:
  - Databricks Runtime 16.4 LTS lub nowszy (zalecane: 17.3 LTS)
  - Unity Catalog włączony
  - Uprawnienia: CREATE TABLE, CREATE SCHEMA, SELECT, MODIFY
  - Klaster: Standard lub **Serverless Compute** (zalecane)
- **Zależności**: 
  - Wykonany notebook `01_delta_lake_operations.ipynb`
  - Wykonany notebook `02_Lakeflow_Connection.ipynb` (dla danych Bronze)
- **Czas realizacji**: ~90 minut

> **Uwaga (2025):** Serverless Compute jest teraz domyślnym trybem dla nowych workloadów.

## Wstęp teoretyczny - Medallion Architecture

**Cel sekcji:** Zrozumienie architektury medalionowej jako fundamentalnego design pattern dla data lakehouse.

---

### Czym jest Medallion Architecture?

**Medallion Architecture** to wielowarstwowy wzorzec organizacji danych w data lakehouse, który dzieli dane na trzy warstwy o rosnącej jakości i wartości biznesowej:

```
DATA SOURCES
    ↓
🥉 BRONZE (Raw)
    ↓ cleansing
🥈 SILVER (Validated)
    ↓ aggregation
🥇 GOLD (Business)
    ↓
CONSUMPTION
```

### Warstwy - Szczegółowy Opis

#### 🥉 Bronze Layer - Raw / Landing Zone

**Charakterystyka:**
- Dane "as-is" bez transformacji wartości
- Append-only, immutable
- Audit metadata: `source_file_path`, `ingestion_ts`, `load_ts`
- Multi-format: JSON, CSV, Parquet, Avro
- Schema-on-read approach

**Retention:** 3-7 lat (długoterminowa historia)

**Use Cases:**
- Data recovery (reprocess pipeline)
- Audit trail & compliance
- Historical analysis
- Data science exploration

**Przykład Bronze Table (nasz model):**
```sql
CREATE TABLE bronze.orders_raw (
    order_id STRING,
    customer_id STRING,
    product_id STRING,
    store_id STRING,
    order_datetime STRING,
    quantity INT,
    unit_price DOUBLE,
    discount_percent INT,
    total_amount DOUBLE,
    payment_method STRING,
    -- Metadata columns
    source_system STRING,           -- 'batch' lub 'stream'
    source_file_path STRING,        -- Ścieżka źródłowa
    ingestion_ts TIMESTAMP,         -- Timestamp przetwarzania
    load_ts TIMESTAMP               -- Timestamp załadowania
)
```

---

#### 🥈 Silver Layer - Cleansed / Validated

**Charakterystyka:**
- **Deduplikacja** po kluczu biznesowym
- **Walidacja**: NOT NULL, data types, ranges (CONSTRAINTS)
- **Kalkulacje biznesowe**: gross_amount, discount_amount, net_amount
- **Flagi jakościowe**: is_return, is_future_dated, is_unknown_*
- **SCD Type 2** dla wymiarów (customers)

**Retention:** 1-2 lata (medium-term history)

**Use Cases:**
- Foundation for analytics
- Joins & enrichment
- ML feature engineering
- Data quality monitoring

**Przykład Silver Table (nasz model):**
```sql
CREATE TABLE silver.orders_clean (
    -- Klucze
    order_id STRING NOT NULL,
    customer_id STRING NOT NULL,
    product_id STRING NOT NULL,
    store_id STRING,
    
    -- Daty
    order_ts TIMESTAMP,
    order_date DATE,
    order_date_key INT,              -- Format: yyyyMMdd (dla joins)
    
    -- Miary oryginalne
    quantity INT,
    unit_price DOUBLE,
    discount_percent INT,
    
    -- Miary kalkulowane
    gross_amount DOUBLE,             -- quantity * unit_price
    discount_amount DOUBLE,          -- gross_amount * discount_percent / 100
    net_amount DOUBLE,               -- gross_amount - discount_amount
    
    -- Flagi jakościowe
    payment_method_code STRING,
    source_system STRING,
    is_return INT,                   -- 1 jeśli quantity < 0
    is_future_dated INT,             -- 1 jeśli order_date > today
    is_unknown_customer INT,
    is_unknown_product INT
)
```

**Constraints (Lakeflow):**
```sql
CONSTRAINT valid_order_id EXPECT (order_id IS NOT NULL) ON VIOLATION DROP ROW
CONSTRAINT valid_quantity EXPECT (quantity IS NOT NULL AND quantity <> 0) ON VIOLATION DROP ROW
CONSTRAINT valid_unit_price EXPECT (unit_price >= 0) ON VIOLATION DROP ROW
```

---

#### 🥇 Gold Layer - Star Schema

**Charakterystyka:**
- **Fact Table**: `fact_sales` - główna tabela transakcji
- **Dimension Tables**: `dim_customer`, `dim_product`, `dim_date`, `dim_store`, `dim_payment_method`
- **Pre-computed joins** (denormalized dla BI)
- **Query-optimized** (partitioned, indexed)

**Retention:** 6-12 miesięcy (refreshable from Silver)

**Use Cases:**
- BI dashboards (Power BI, Tableau)
- Executive reports
- ML model training
- Self-service analytics

**Przykład Gold Tables (nasz model - Star Schema):**

```sql
-- FACT TABLE
CREATE TABLE gold.fact_sales (
    order_id STRING,
    customer_id STRING,
    product_id STRING,
    store_id STRING,
    payment_method_code STRING,
    order_date_key INT,              -- FK do dim_date
    order_ts TIMESTAMP,
    
    -- Miary
    quantity INT,
    unit_price DOUBLE,
    discount_percent INT,
    gross_amount DOUBLE,
    discount_amount DOUBLE,
    net_amount DOUBLE,
    
    -- Flagi
    is_return INT,
    is_future_dated INT,
    source_system STRING
)

-- DIMENSION TABLES
CREATE TABLE gold.dim_customer (
    customer_id STRING,
    first_name STRING,
    last_name STRING,
    email STRING,
    city STRING,
    country STRING,
    customer_segment STRING
)

CREATE TABLE gold.dim_date (
    date_key INT,                    -- PK: yyyyMMdd
    date DATE,
    year INT,
    quarter INT,
    month INT,
    day INT,
    day_of_week STRING,
    is_weekend INT
)

CREATE TABLE gold.dim_payment_method (
    payment_method_code STRING,
    payment_method_group STRING      -- Card, Cash, Digital wallet
)
```

---

### Kluczowe Zasady Medallion Architecture

**1. Separation of Concerns**
- Bronze: Ingestion (raw data)
- Silver: Data Quality (cleansing, validation, kalkulacje)
- Gold: Business Logic (Star Schema, agregacje)

**2. Incremental Processing**
- Process tylko nowe/zmienione dane
- Delta Lake MERGE operations
- Checkpoint management dla streaming

**3. Idempotency**
- Można uruchomić wielokrotnie bez duplikacji
- Deterministic transformations
- Unique keys & deduplication

**4. Schema Evolution**
- Bronze: Flexible (rescued data)
- Silver: Controlled (addNewColumns)
- Gold: Strict (versioned)

**5. Data Quality Gates**
- CONSTRAINTS w Silver (DROP ROW on violation)
- Flagi jakościowe (is_return, is_future_dated)
- UNKNOWN handling (dim_product z is_unknown=1)

---

### ETL vs ELT w Medallion

**Traditional ETL:**
```
Extract → Transform → Load
         (outside DB)
```

**Medallion ELT:**
```
Extract → Load (Bronze) → Transform (Silver) → Load (Gold)
                 ↓                    ↓              ↓
             raw data          cleansed data    star schema
```

**Dlaczego ELT?**
- Zachowanie raw data (compliance)
- Flexibility (re-transform later)
- Scalability (Spark distributed processing)
- Cost-effective (storage cheaper than compute)

---

### Medallion vs Traditional Data Warehouse

| Feature | Traditional DWH | Medallion Lakehouse |
|---------|-----------------|---------------------|
| **Storage** | Proprietary (expensive) | Cloud object storage (cheap) |
| **Schema** | Schema-on-write | Schema-on-read (Bronze) |
| **Data Types** | Structured only | Structured + semi-structured |
| **Flexibility** | Rigid | Flexible (schema evolution) |
| **Raw Data** | Discarded | Preserved (Bronze) |
| **Processing** | ETL (batch) | ELT (batch + streaming) |
| **Cost** | High (compute + storage) | Lower (decouple compute/storage) |
| **Use Cases** | BI & reporting | BI + ML + data science |

---

### Best Practices

**1. Naming Conventions (nasz model):**
```
bronze.{entity}_raw           -- bronze.orders_raw
silver.{entity}_clean         -- silver.orders_clean
gold.fact_{entity}            -- gold.fact_sales
gold.dim_{dimension}          -- gold.dim_customer
```

**2. Partitioning:**
- Bronze: Ingestion date (`ingestion_ts`)
- Silver: Business date (`order_date`)
- Gold: Report date (`order_date_key`)

**3. Metadata Columns:**
```python
# Bronze (nasz model)
source_file_path, ingestion_ts, load_ts, source_system

# Silver (nasz model)
order_date_key, is_return, is_future_dated, is_unknown_*

# Gold (nasz model)
order_date_key (surrogate key for joins)
```

**4. Refresh Cadence:**
- Bronze: Real-time (Lakeflow STREAMING TABLE)
- Silver: Real-time (Lakeflow STREAMING TABLE)
- Gold: Real-time (Lakeflow MATERIALIZED VIEW)

---

### Pipeline Architecture Diagram (nasz model)

```
┌──────────────────────────────────────────────────────────────┐
│                      DATA SOURCES                             │
│  • customers.csv  • orders_batch.json  • products.parquet    │
│  • orders_stream_*.json (streaming)                          │
└────────────────────────┬─────────────────────────────────────┘
                         │ Lakeflow: read_files() / STREAM
                         ▼
┌──────────────────────────────────────────────────────────────┐
│                   🥉 BRONZE LAYER                             │
│                                                               │
│  bronze.customers_raw      bronze.orders_raw                 │
│  bronze.products_raw                                         │
│                                                               │
│  Features:                                                    │
│  • Raw data (as-is)                                          │
│  • Metadata: source_file_path, ingestion_ts, load_ts         │
│  • source_system: 'batch' / 'stream'                         │
└────────────────────────┬─────────────────────────────────────┘
                         │ MERGE / CONSTRAINTS / Calculations
                         ▼
┌──────────────────────────────────────────────────────────────┐
│                   🥈 SILVER LAYER                             │
│                                                               │
│  silver.customers_type2    (SCD Type 2: __START_AT, __END_AT)│
│  silver.orders_clean       (gross/discount/net_amount)       │
│  silver.products                                             │
│                                                               │
│  Features:                                                    │
│  • Calculated: gross_amount, discount_amount, net_amount     │
│  • Flags: is_return, is_future_dated, is_unknown_*           │
│  • CONSTRAINTS: valid_order_id, valid_quantity, etc.         │
└────────────────────────┬─────────────────────────────────────┘
                         │ Star Schema / Dimensional Model
                         ▼
┌──────────────────────────────────────────────────────────────┐
│                   🥇 GOLD LAYER (Star Schema)                 │
│                                                               │
│  FACT: fact_sales                                            │
│  DIMS: dim_customer, dim_product, dim_date,                  │
│        dim_store, dim_payment_method                         │
│                                                               │
│  Features:                                                    │
│  • Surrogate keys (order_date_key = yyyyMMdd)                │
│  • Denormalized dimensions                                    │
│  • UNKNOWN handling (is_unknown flag)                        │
└────────────────────────┬─────────────────────────────────────┘
                         │
                         ▼
┌──────────────────────────────────────────────────────────────┐
│                  CONSUMPTION LAYER                            │
│  • Power BI  • Tableau  • SQL Analytics  • ML Models         │
│                                                               │
│  Example Query:                                               │
│  SELECT f.*, d.year, c.customer_segment, p.product_name      │
│  FROM fact_sales f                                           │
│  JOIN dim_date d ON f.order_date_key = d.date_key            │
│  JOIN dim_customer c ON f.customer_id = c.customer_id        │
│  JOIN dim_product p ON f.product_id = p.product_id           │
└──────────────────────────────────────────────────────────────┘
```

## Izolacja per użytkownik

Uruchom skrypt inicjalizacyjny dla per-user izolacji katalogów i schematów:

In [0]:
%run ../00_setup

## Konfiguracja

Import bibliotek i ustawienie zmiennych środowiskowych:

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from datetime import datetime, timedelta
import time

# Wyświetl kontekst użytkownika
display({
    "Katalog": CATALOG,
    "Schema Bronze": BRONZE_SCHEMA,
    "Schema Silver": SILVER_SCHEMA,
    "Schema Gold": GOLD_SCHEMA,
    "Użytkownik": raw_user,
    "Dataset base": DATASET_BASE_PATH
})

# Ustaw katalog i schemat jako domyślne
spark.sql(f"USE CATALOG {CATALOG}")

### Kontekst użytkownika

Wyświetlenie aktualnej konfiguracji środowiska oraz ścieżek do danych:

In [0]:
spark.sql(f"USE CATALOG {CATALOG}")

**Konfiguracja katalogu Unity Catalog:**

Ustawienie domyślnego katalogu dla wszystkich operacji.

## Sekcja 1: Bronze Layer - Raw Data Landing

**Cel sekcji:** Zrozumienie roli Bronze layer jako landing zone dla raw data.

### Bronze Layer - Kluczowe Cechy

**1. Raw Data "As-Is"**
- Dane zapisywane bez transformacji wartości
- Zachowanie oryginalnego formatu
- Multi-format support (JSON, CSV, Parquet)

**2. Append-Only Pattern**
- Nigdy nie usuwamy/modyfikujemy danych
- Immutable history
- Time-travel capability

**3. Audit Metadata**
```python
# Metadane audytowe w Bronze (zgodne z Lakeflow)
source_file_path     # Ścieżka źródłowa pliku
ingestion_ts         # Timestamp przetwarzania pliku
load_ts              # Timestamp załadowania do Bronze
```

**4. Schema-on-Read**
- Elastyczny schemat (może się zmieniać)
- Rescued data column dla unknown columns
- Reprocessing capability

### Bronze Tables - Model Danych (zgodny z Lakeflow)

**`bronze_customers`** (CSV):
- `customer_id`, `first_name`, `last_name`, `email`, `phone`
- `city`, `state`, `country`, `registration_date`, `customer_segment`
- Metadata: `source_file_path`, `ingestion_ts`, `load_ts`

**`bronze_orders`** (JSON):
- `order_id`, `customer_id`, `product_id`, `store_id`
- `order_datetime`, `quantity`, `unit_price`, `discount_percent`, `total_amount`
- `payment_method`, `source_system`
- Metadata: `source_file_path`, `ingestion_ts`, `load_ts`

**`bronze_products`** (Parquet):
- `product_id`, `product_name`, `subcategory_code`, `brand`
- `unit_cost`, `list_price`, `weight_kg`, `status`
- Metadata: `source_file_path`, `ingestion_ts`, `load_ts`

### Dlaczego Bronze jest Ważny?

**1. Data Recovery**
```python
# Możemy reprocessować pipeline od Bronze
bronze_data = spark.table("bronze.orders_raw")
# Re-run transformations → Silver → Gold
```

**2. Schema Evolution**
```python
# Nowe kolumny w source nie łamią pipeline
# Trafiają do _rescued_data
```

**3. Compliance & Audit**
```python
# Pełna historia: kto, co, kiedy załadował
# Retention: 3-7 lat (regulacje prawne)
```

**4. Data Science Exploration**
```python
# Analitycy mogą eksplorować raw data
# Tworzyć nowe features z surowych danych
```

### Przykład 1.1: Inspekcja Bronze Layer

**Cel:** Sprawdzić dane w Bronze layer i zrozumieć ich strukturę.

### Przykład 1.1: Ingestion do Bronze Layer (Raw Data)

Wczytujemy dane bezpośrednio z plików źródłowych (CSV, JSON, Parquet) do tabel Bronze.
Dzięki temu notebook jest niezależny od poprzednich kroków.

In [0]:
# 1. Load Customers (CSV) - zgodne z modelem Lakeflow
customers_path = f"{DATASET_BASE_PATH}/customers/customers.csv"

customers_df = (spark.read
    .format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(customers_path)
    # Dodaj metadata columns (jak w Lakeflow)
  #  .withColumn("source_file_path", F.input_file_name())
    .withColumn("ingestion_ts", F.current_timestamp())
    .withColumn("load_ts", F.current_timestamp())
)

display(customers_df.limit(5))

In [0]:
# 3. Load Products (Parquet) - zgodne z modelem Lakeflow
products_path = f"{DATASET_BASE_PATH}/products/products.parquet"

products_df = (spark.read
    .format("parquet")
    .load(products_path)
    # Dodaj metadata columns (jak w Lakeflow)
 #   .withColumn("source_file_path", F.input_file_name())
    .withColumn("ingestion_ts", F.current_timestamp())
    .withColumn("load_ts", F.current_timestamp())
)

# Write to Bronze
(products_df.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable(f"{BRONZE_SCHEMA}.products_raw")
)

display({"status": f"✅ Table {BRONZE_SCHEMA}.products_raw created/overwritten"})

In [0]:
# 2. Load Orders (JSON) - zgodne z modelem Lakeflow
orders_path = f"{DATASET_BASE_PATH}/orders/orders_batch.json"

orders_df = (spark.read
    .format("json")
    .load(orders_path)
    # Dodaj metadata columns (jak w Lakeflow)
    .withColumn("source_system", F.lit("batch"))
    #.withColumn("source_file_path", F.input_file_name())
    .withColumn("ingestion_ts", F.current_timestamp())
    .withColumn("load_ts", F.current_timestamp())
)

# Write to Bronze
(orders_df.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable(f"{BRONZE_SCHEMA}.orders_raw")
)

display({"status": f"✅ Table {BRONZE_SCHEMA}.orders_raw created/overwritten"})

In [0]:
# Write Customers to Bronze
(customers_df.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable(f"{BRONZE_SCHEMA}.customers_raw")
)

display({"status": f"✅ Table {BRONZE_SCHEMA}.customers_raw created/overwritten"})

**Utworzone tabele Bronze:**

Konwersja danych z batch do raw tables została zakończona pomyślnie.

In [0]:
# Inspekcja Bronze Layer
bronze_tables = ["customers_raw", "orders_raw", "products_raw"]
results = []

for table in bronze_tables:
    full_table = f"{CATALOG}.{BRONZE_SCHEMA}.{table}"
    
    if spark.catalog.tableExists(full_table):
        df = spark.table(full_table)
        results.append({
            "table": table,
            "status": "✅",
            "records": df.count(),
            "columns": len(df.columns)
        })

display(spark.createDataFrame(results))

**Podsumowanie inspekcji Bronze Layer:**

Bronze Layer zawiera RAW data bez transformacji, zachowuje pełną historię (append-only) i stanowi foundation dla dalszych transformacji.

## Sekcja 2: Silver Layer - Cleansing & Validation

**Cel sekcji:** Transformacja danych z Bronze do Silver z zastosowaniem data quality rules.

### Silver Layer - Model Danych (zgodny z Lakeflow)

**`silver_orders`** - Oczyszczone i policzone miary:
- Klucze: `order_id`, `customer_id`, `product_id`, `store_id`
- Daty: `order_ts`, `order_date`, `order_date_key` (INT: yyyyMMdd)
- Miary oryginalne: `quantity`, `unit_price`, `discount_percent`
- **Miary kalkulowane:**
  - `gross_amount = quantity * unit_price`
  - `discount_amount = quantity * unit_price * discount_percent / 100`
  - `net_amount = gross_amount - discount_amount`
- `payment_method_code` (standaryzowane)
- **Flagi jakościowe:**
  - `is_return` (quantity < 0 OR total_amount < 0)
  - `is_future_dated` (order_date > current_date)
  - `is_unknown_customer`, `is_unknown_product`

**`silver_customers`** (SCD Type 2):
- Pola: `customer_id`, `first_name`, `last_name`, `email`, `phone`, `city`, `state`, `country`
- SCD2 columns: `__START_AT`, `__END_AT` (lub `effective_from`, `effective_to`)

**`silver_products`**:
- `product_id`, `product_name`, `subcategory_code`, `brand`
- `unit_cost`, `list_price`, `weight_kg`, `status`
- `is_active`, `is_unknown`

### Constraints (jak w Lakeflow)

```sql
CONSTRAINT valid_order_id EXPECT (order_id IS NOT NULL) ON VIOLATION DROP ROW
CONSTRAINT valid_customer EXPECT (customer_id IS NOT NULL) ON VIOLATION DROP ROW
CONSTRAINT valid_product EXPECT (product_id IS NOT NULL) ON VIOLATION DROP ROW
CONSTRAINT valid_quantity EXPECT (quantity IS NOT NULL AND quantity <> 0) ON VIOLATION DROP ROW
CONSTRAINT valid_unit_price EXPECT (unit_price IS NOT NULL AND unit_price >= 0) ON VIOLATION DROP ROW
```

### MERGE Operation - Deduplication Pattern

**Problem:** Bronze zawiera duplikaty (append-only)

**Rozwiązanie:** MERGE w Silver (upsert)

```sql
MERGE INTO silver.orders_clean AS target
USING (
    SELECT DISTINCT *
    FROM bronze.orders_raw
    WHERE ingestion_ts > (
        SELECT MAX(load_ts) FROM silver.orders_clean
    )
) AS source
ON target.order_id = source.order_id
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *
```

### Przykład 2.1: Bronze → Silver Transformation (Orders)

**Cel:** Transform orders z Bronze do Silver z cleansing i validation.

In [0]:
# Bronze → Silver: Orders (zgodne z modelem Lakeflow)
bronze_orders = spark.table(f"{BRONZE_SCHEMA}.orders_raw").dropDuplicates(["customer_id"])
bronze_orders.write.format("delta").mode("overwrite").saveAsTable(f"{BRONZE_SCHEMA}.orders_raw")

In [0]:
# Bronze → Silver: Orders (zgodne z modelem Lakeflow)
bronze_orders = spark.table(f"{BRONZE_SCHEMA}.orders_raw")

# Transform & Validate - kalkulacje jak w Lakeflow silver_orders
silver_orders = (bronze_orders
    # Klucze
    .withColumn("order_id", F.col("order_id"))
    .withColumn("customer_id", F.col("customer_id"))
    .withColumn("product_id", F.col("product_id"))
    .withColumn("store_id", F.col("store_id"))
    
    # Daty
    .withColumn("order_ts", F.to_timestamp(F.col("order_datetime")))
    .withColumn("order_date", F.to_date(F.col("order_datetime")))
    .withColumn("order_date_key", F.date_format(F.col("order_datetime"), "yyyyMMdd").cast("int"))
    
    # Miary oryginalne
    .withColumn("quantity", F.col("quantity").cast("int"))
    .withColumn("unit_price", F.col("unit_price").cast("double"))
    .withColumn("discount_percent", F.col("discount_percent").cast("int"))
    
    # Miary kalkulowane (zgodne z Lakeflow)
    .withColumn("gross_amount", F.col("quantity") * F.col("unit_price"))
    .withColumn("discount_amount", F.col("quantity") * F.col("unit_price") * F.col("discount_percent") / 100.0)
    .withColumn("net_amount", 
        F.col("gross_amount") - F.col("discount_amount")
    )
    
    # Payment method (standaryzowane)
    .withColumn("payment_method_code", F.coalesce(F.col("payment_method"), F.lit("Unknown")))
    .withColumn("source_system", F.col("source_system"))
    
    # Flagi jakościowe (zgodne z Lakeflow)
    .withColumn("is_return", 
        F.when((F.col("quantity") < 0) | (F.col("total_amount") < 0), 1).otherwise(0)
    )
    .withColumn("is_future_dated",
        F.when(F.col("order_date") > F.current_date(), 1).otherwise(0)
    )
    .withColumn("is_unknown_customer",
        F.when(F.col("customer_id").isNull() | (F.col("customer_id") == "CUST999999"), 1).otherwise(0)
    )
    .withColumn("is_unknown_product",
        F.when(F.col("product_id").isNull() | (F.col("product_id") == "PROD999999"), 1).otherwise(0)
    )
    
    # Validation - DROP invalid rows (jak CONSTRAINT w Lakeflow)
    .filter(F.col("order_id").isNotNull())
    .filter(F.col("customer_id").isNotNull())
    .filter(F.col("product_id").isNotNull())
    .filter((F.col("quantity").isNotNull()) & (F.col("quantity") != 0))
    .filter((F.col("unit_price").isNotNull()) & (F.col("unit_price") >= 0))
    
    # Select final columns
    .select(
        "order_id", "customer_id", "product_id", "store_id",
        "order_ts", "order_date", "order_date_key",
        "quantity", "unit_price", "discount_percent",
        "gross_amount", "discount_amount", "net_amount",
        "payment_method_code", "source_system",
        "is_return", "is_future_dated", "is_unknown_customer", "is_unknown_product"
    )
)

# Write to Silver
silver_orders.write.format("delta").mode("overwrite").saveAsTable(f"{SILVER_SCHEMA}.orders_clean")

# Stats
bronze_count = bronze_orders.count()
silver_count = silver_orders.count()
dropped_count = bronze_count - silver_count

display({
    "bronze_records": bronze_count,
    "silver_records": silver_count,
    "dropped_by_constraints": dropped_count,
    "status": f"✅ Created {SILVER_SCHEMA}.orders_clean"
})

In [0]:
# Podgląd Silver Orders z nowymi kalkulacjami
display(
    spark.table(f"{SILVER_SCHEMA}.orders_clean")
    .select("order_id", "order_date", "quantity", "unit_price", "discount_percent", 
            "gross_amount", "discount_amount", "net_amount", "is_return", "is_future_dated")
    .limit(10)
)

**Uwaga dotycząca production:**

W środowisku produkcyjnym użylibyśmy operacji MERGE dla deduplikacji zamiast prostego overwrite.

## Sekcja 3: SCD (Slowly Changing Dimensions)

**Cel sekcji:** Implementacja SCD Type 1 i Type 2 dla śledzenia zmian w danych.

### Model SCD w Lakeflow

W naszym Lakeflow pipeline, `silver_customers` używa **SCD Type 2** z kolumnami:
- `__START_AT` - timestamp początku ważności rekordu
- `__END_AT` - timestamp końca ważności (NULL = aktualny rekord)

```sql
-- Lakeflow AUTO CDC for SCD2
CREATE FLOW silver_customers_scd2_flow
AS AUTO CDC INTO silver_customers
FROM STREAM bronze_customers
KEYS (customer_id)
SEQUENCE BY ingestion_ts
STORED AS SCD TYPE 2;
```

### SCD Types - Overview

| Type | Strategy | History | Use Case |
|------|----------|---------|----------|
| **Type 0** | No changes allowed | N/A | Reference data (countries) |
| **Type 1** | Overwrite | ❌ No | Current state only |
| **Type 2** | Add new row | ✅ Yes | Full history tracking |
| **Type 3** | Add new column | ⚠️ Limited | Previous value only |

---

### SCD Type 1 - Overwrite

**Strategia:** Nadpisz starą wartość nową (bez historii)

**Implementacja:** Simple UPDATE/MERGE

**Pros:**
- ✅ Simple implementation
- ✅ No history bloat
- ✅ Always current values

**Cons:**
- ❌ No historical tracking
- ❌ Can't analyze "as of date"
- ❌ Lose audit trail

**Use Cases:**
- Correcting data entry errors
- Non-critical attributes (e.g., marketing preferences)

---

### SCD Type 2 - Historical Tracking (używane w Lakeflow)

**Strategia:** Dodaj nowy rekord dla każdej zmiany (pełna historia)

**Kolumny SCD Type 2 (Lakeflow format):**
- `__START_AT`: Timestamp początku ważności
- `__END_AT`: Timestamp końca ważności (NULL = current)

**Query aktualne rekordy:**
```sql
SELECT * FROM silver_customers WHERE __END_AT IS NULL
```

**Query historyczne (as of date):**
```sql
SELECT * FROM silver_customers 
WHERE '2024-01-15' >= __START_AT 
  AND ('2024-01-15' < __END_AT OR __END_AT IS NULL)
```

**Use Cases:**
- Customer dimensions (address, preferences)
- Product dimensions (price history)
- Compliance & audit requirements

### Przykład 3.1: SCD Type 1 - Customers (Overwrite)

**Cel:** Implementacja SCD Type 1 - prosty overwrite bez historii.

In [0]:
# SCD Type 1: Customers (Current State Only)
bronze_customers = spark.table(f"{BRONZE_SCHEMA}.customers_raw")

# Transform to Silver (SCD Type 1 - current state only)
customers_type1 = (bronze_customers
    .withColumn("customer_id", F.col("customer_id"))
    .withColumn("first_name", F.trim(F.col("first_name")))
    .withColumn("last_name", F.trim(F.col("last_name")))
    .withColumn("email", F.lower(F.trim(F.col("email"))))
    .withColumn("phone", F.col("phone"))
    .withColumn("city", F.initcap(F.trim(F.col("city"))))
    .withColumn("state", F.upper(F.trim(F.col("state"))))
    .withColumn("country", F.upper(F.trim(F.col("country"))))
    .withColumn("registration_date", F.to_date(F.col("registration_date")))
    .withColumn("customer_segment", F.col("customer_segment"))
    .withColumn("updated_at", F.current_timestamp())
    .select(
        "customer_id", "first_name", "last_name", "email", "phone",
        "city", "state", "country", "registration_date", "customer_segment",
        "updated_at"
    )
   # .dropDuplicates(["customer_id"])  # Remove duplicates by customer_id
)

# Create/Replace table (Type 1 = overwrite)
customers_type1.write.format("delta").mode("overwrite").saveAsTable(f"{SILVER_SCHEMA}.customers_type1")

display({
    "status": f"✅ Created {SILVER_SCHEMA}.customers_type1",
    "records": customers_type1.count(),
    "note": "SCD Type 1: Zawsze aktualny stan, bez historii"
})

**Utworzona tabela SCD Type 1:**

Tabela `customers_type1` zawiera zawsze aktualny stan bez historii. Wyświetlenie przykładowych danych:

In [0]:
display(spark.table(f"{SILVER_SCHEMA}.customers_type1").limit(5))

### Symulacja UPDATE - zmiana miasta dla customer_id=1

**PRZED ZMIANĄ:** Wyświetlenie aktualnego stanu customer_id=1:

In [0]:
display(
    spark.sql(f"""
        SELECT customer_id, first_name, last_name, city, state, country, customer_segment
        FROM {SILVER_SCHEMA}.customers_type1 
        WHERE customer_id = 'CUST000001'
    """)
)

In [0]:
# Symulacja UPDATE - zmiana miasta dla customer_id
from pyspark.sql.types import StructType, StructField, StringType

updates_schema = StructType([
    StructField("customer_id", StringType(), True),
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("email", StringType(), True),
    StructField("phone", StringType(), True),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True),
    StructField("country", StringType(), True),
    StructField("registration_date", StringType(), True),
    StructField("customer_segment", StringType(), True)
])

updates_data = [("CUST000001", "Jesse", "Hoffman", "jesse.hoffman@interia.pl", "+48 694 026 542", 
                 "Kraków", "MA", "POL", "2025-10-16", "Premium")]  # Changed city & country!
updates_df = spark.createDataFrame(updates_data, updates_schema)

In [0]:
# SCD Type 1: MERGE (Overwrite)
updates_df.createOrReplaceTempView("customer_updates")

spark.sql(f"""
    MERGE INTO {SILVER_SCHEMA}.customers_type1 AS target
    USING customer_updates AS source
    ON target.customer_id = source.customer_id
    WHEN MATCHED THEN UPDATE SET
        target.city = source.city,
        target.state = source.state,
        target.country = source.country,
        target.customer_segment = source.customer_segment,
        target.updated_at = current_timestamp()
    WHEN NOT MATCHED THEN INSERT (
        customer_id, first_name, last_name, email, phone,
        city, state, country, registration_date, customer_segment, updated_at
    ) VALUES (
        source.customer_id, source.first_name, source.last_name, source.email, source.phone,
        source.city, source.state, source.country, CAST(source.registration_date AS DATE), 
        source.customer_segment, current_timestamp()
    )
""")

**PO ZMIANIE (SCD Type 1 - overwrite):** Wyświetlenie zaktualizowanego stanu:

In [0]:
display(
    spark.sql(f"""
        SELECT customer_id, first_name, last_name, city, state, country, customer_segment
        FROM {SILVER_SCHEMA}.customers_type1 
        WHERE customer_id = 'CUST000001'
    """)
)

**⚠️  UWAGA:** Historia zmiany została UTRACONA

**💡 Stara wartość (Warszawa) została nadpisana (Kraków)**

### Przykład 3.2: SCD Type 2 - Customers (Historical Tracking)

**Cel:** Implementacja SCD Type 2 - pełne śledzenie historii zmian.

In [0]:
# SCD Type 2: Customers (Historical Tracking - format Lakeflow)
bronze_customers = spark.table(f"{BRONZE_SCHEMA}.customers_raw")

# Transform with SCD Type 2 columns (Lakeflow format: __START_AT, __END_AT)
customers_type2_initial = (bronze_customers
    .withColumn("customer_id", F.col("customer_id"))
    .withColumn("first_name", F.trim(F.col("first_name")))
    .withColumn("last_name", F.trim(F.col("last_name")))
    .withColumn("email", F.lower(F.trim(F.col("email"))))
    .withColumn("phone", F.col("phone"))
    .withColumn("city", F.initcap(F.trim(F.col("city"))))
    .withColumn("state", F.upper(F.trim(F.col("state"))))
    .withColumn("country", F.upper(F.trim(F.col("country"))))
    .withColumn("registration_date", F.to_date(F.col("registration_date")))
    .withColumn("customer_segment", F.col("customer_segment"))
    # SCD Type 2 columns (Lakeflow format)
    .withColumn("__START_AT", F.current_timestamp())
    .withColumn("__END_AT", F.lit(None).cast("timestamp"))  # NULL = current record
    .select(
        "customer_id", "first_name", "last_name", "email", "phone",
        "city", "state", "country", "registration_date", "customer_segment",
        "__START_AT", "__END_AT"
    )
)

# Create initial table
customers_type2_initial.write.format("delta").mode("overwrite").saveAsTable(f"{SILVER_SCHEMA}.customers_type2")

display({
    "status": f"✅ Created {SILVER_SCHEMA}.customers_type2",
    "records": customers_type2_initial.count(),
    "columns": "__START_AT, __END_AT (Lakeflow format)"
})

**Utworzona tabela SCD Type 2:**

Tabela `customers_type2` zawiera kolumny: `__START_AT`, `__END_AT` (format Lakeflow). Wyświetlenie przykładowych danych:

In [0]:
display(spark.table(f"{SILVER_SCHEMA}.customers_type2").limit(5))

### Symulacja CHANGE - zmiana miasta dla customer_id=1

**PRZED ZMIANĄ:** Wyświetlenie aktualnego stanu customer_id=1:

In [0]:
display(
    spark.sql(f"""
        SELECT customer_id, first_name, last_name, city, country, __START_AT, __END_AT
        FROM {SILVER_SCHEMA}.customers_type2 
        WHERE customer_id = 'CUST000001'
        ORDER BY __START_AT
    """)
)

In [0]:
# Symulacja UPDATE dla SCD Type 2 - ten sam update co dla Type 1
# Używamy tego samego customer_updates view (już utworzony wcześniej)

**Step 1: Close old records** (set effective_to, is_current=false):

In [0]:
# SCD Type 2: Step 1 - Close old records (set __END_AT)
spark.sql(f"""
    MERGE INTO {SILVER_SCHEMA}.customers_type2 AS target
    USING (
        SELECT DISTINCT u.customer_id
        FROM customer_updates u
        INNER JOIN {SILVER_SCHEMA}.customers_type2 t
            ON u.customer_id = t.customer_id
        WHERE t.__END_AT IS NULL
          AND (u.city != t.city OR u.country != t.country OR u.customer_segment != t.customer_segment)
    ) AS changed
    ON target.customer_id = changed.customer_id 
       AND target.__END_AT IS NULL
    WHEN MATCHED THEN UPDATE SET
        target.__END_AT = current_timestamp()
""")

**Step 2: Insert new versions** - dodanie nowych rekordów z updated values:

In [0]:
# SCD Type 2: Step 2 - Insert new versions
spark.sql(f"""
    INSERT INTO {SILVER_SCHEMA}.customers_type2
    SELECT 
        u.customer_id,
        u.first_name,
        u.last_name,
        u.email,
        u.phone,
        u.city,
        u.state,
        u.country,
        CAST(u.registration_date AS DATE),
        u.customer_segment,
        current_timestamp() AS __START_AT,
        CAST(NULL AS TIMESTAMP) AS __END_AT
    FROM customer_updates u
    WHERE NOT EXISTS (
        SELECT 1 FROM {SILVER_SCHEMA}.customers_type2 existing
        WHERE existing.customer_id = u.customer_id
          AND existing.__END_AT IS NULL
          AND existing.city = u.city
          AND existing.country = u.country
          AND existing.customer_segment = u.customer_segment
    )
""")

**PO ZMIANIE (SCD Type 2 - historical tracking):**

Historia została zachowana! Wyświetlenie wszystkich wersji customer_id=1:

In [0]:
# Wyświetl historię zmian dla CUST000001
display(
    spark.sql(f"""
        SELECT 
            customer_id,
            first_name,
            city,
            country,
            customer_segment,
            __START_AT,
            __END_AT,
            CASE WHEN __END_AT IS NULL THEN 'CURRENT' ELSE 'HISTORICAL' END AS status
        FROM {SILVER_SCHEMA}.customers_type2 
        WHERE customer_id = 'CUST000001'
        ORDER BY __START_AT
    """)
)

**✅ Historia zachowana! (format Lakeflow)**

Mamy teraz 2 rekordy:
- **Record 1**: New York, USA (__END_AT = timestamp zamknięcia)
- **Record 2**: Kraków, POL (__END_AT = NULL = aktualny rekord)

W Lakeflow ten sam efekt uzyskujemy automatycznie przez `AUTO CDC ... STORED AS SCD TYPE 2`.

### Przykład: Query 'as of date'

**Gdzie mieszkał klient 1 miesiąc temu?** SCD Type 2 umożliwia temporal queries:

In [0]:
one_month_ago = (datetime.now() - timedelta(days=30)).date()

In [0]:
# Query "as of date" - format Lakeflow
display(
    spark.sql(f"""
        SELECT 
            customer_id,
            first_name,
            city,
            country,
            __START_AT,
            __END_AT
        FROM {SILVER_SCHEMA}.customers_type2
        WHERE customer_id = 'CUST000001'
          AND '{one_month_ago}' >= __START_AT 
          AND ('{one_month_ago}' < __END_AT OR __END_AT IS NULL)
    """)
)

## Sekcja 4: Gold Layer - Star Schema (zgodny z Lakeflow)

**Cel sekcji:** Implementacja Star Schema zgodnie z modelem Lakeflow.

### Gold Layer - Model Danych (Lakeflow)

**Fact Table:**
- `fact_sales` - główna tabela faktów z miarami transakcyjnymi

**Dimension Tables:**
- `dim_customer` - wymiar klientów (snapshot z SCD2)
- `dim_product` - wymiar produktów
- `dim_date` - wymiar czasu
- `dim_store` - wymiar sklepów
- `dim_payment_method` - wymiar metod płatności

### Star Schema Diagram

```
                    ┌─────────────────┐
                    │   dim_date      │
                    │ date_key (PK)   │
                    │ year, quarter   │
                    │ month, day      │
                    │ is_weekend      │
                    └────────┬────────┘
                             │
┌──────────────┐    ┌────────┴────────┐    ┌──────────────┐
│ dim_customer │    │   fact_sales    │    │ dim_product  │
│ customer_id  │────│ order_id (PK)   │────│ product_id   │
│ first_name   │    │ customer_id (FK)│    │ product_name │
│ last_name    │    │ product_id (FK) │    │ unit_cost    │
│ city, state  │    │ store_id (FK)   │    │ list_price   │
│ segment      │    │ date_key (FK)   │    │ is_active    │
└──────────────┘    │ payment_method  │    └──────────────┘
                    │ quantity        │
                    │ unit_price      │
                    │ gross_amount    │
                    │ discount_amount │
                    │ net_amount      │
                    │ is_return       │
                    └────────┬────────┘
                             │
              ┌──────────────┴──────────────┐
              │                             │
    ┌─────────┴─────────┐     ┌─────────────┴─────────┐
    │    dim_store      │     │ dim_payment_method    │
    │ store_id          │     │ payment_method_code   │
    │ store_code        │     │ payment_method_group  │
    └───────────────────┘     └───────────────────────┘
```

### Przykład 4.1: dim_customer

In [0]:


display(spark.table(f"""{GOLD_SCHEMA}.dim_customer""" ))

In [0]:
# Gold: dim_customer (snapshot z SCD2 - tylko aktualne rekordy)
dim_customer = spark.sql(f"""
    SELECT
        customer_id,
        first_name,
        last_name,
        email,
        phone,
        city,
        state,
        country,
        registration_date,
        customer_segment
    FROM {SILVER_SCHEMA}.customers_type1
  --  WHERE status = 'CURRENT'
""")

dim_customer.write.format("delta").mode("overwrite").saveAsTable(f"{GOLD_SCHEMA}.dim_customer")

display({
    "table": f"{GOLD_SCHEMA}.dim_customer",
    "records": dim_customer.count(),
    "status": "✅ Created"
})

In [0]:
from pyspark.sql import functions as F

display(
    spark.table(f"{GOLD_SCHEMA}.dim_customer")
    .agg(
        F.count("customer_id").alias("all_customer_count"),
        F.count_distinct("customer_id").alias("distinct_customer_count")
    )
)

**Utworzona tabela dim_customer:**

In [0]:
display(spark.table(f"{GOLD_SCHEMA}.dim_customer").limit(5))

### Przykład 4.2: dim_product (zgodne z Lakeflow)

**Utworzona tabela dim_product:**

In [0]:
# Gold: dim_product (zgodne z Lakeflow silver_products)
from pyspark.sql.types import (
    StringType, DoubleType, IntegerType, StructType, StructField
)

bronze_products = spark.table(f"{BRONZE_SCHEMA}.products_raw")

dim_product = (bronze_products
    .withColumn("unit_cost", F.col("unit_cost").cast("double"))
    .withColumn("list_price", F.col("list_price").cast("double"))
    .withColumn("weight_kg", F.col("weight_kg").cast("double"))
    .withColumn("is_active", 
        F.when(F.upper(F.col("status")).isin("ACTIVE", "AVAILABLE"), 1).otherwise(0)
    )
    .withColumn("is_unknown", F.lit(0))
    .select(
        "product_id", "product_name", "subcategory_code", "brand",
        "unit_cost", "list_price", "weight_kg", "status",
        "is_active", "is_unknown"
    )
)


unknown_schema = StructType([
    StructField("product_id", StringType(), False),
    StructField("product_name", StringType(), False),
    StructField("subcategory_code", StringType(), False),
    StructField("brand", StringType(), False),
    StructField("unit_cost", DoubleType(), True),
    StructField("list_price", DoubleType(), True),
    StructField("weight_kg", DoubleType(), True),
    StructField("status", StringType(), False),
    StructField("is_active", IntegerType(), False),
    StructField("is_unknown", IntegerType(), False)
])

unknown_product = spark.createDataFrame([
    ("UNKNOWN", "Unknown product", "UNKNOWN", "Unknown", None, None, None, "Unknown", 0, 1)
], schema=unknown_schema)

dim_product_final = dim_product.unionByName(unknown_product)

dim_product_final.write.format("delta").mode("overwrite").saveAsTable(
    f"{GOLD_SCHEMA}.dim_product"
)

display({
    "table": f"{GOLD_SCHEMA}.dim_product",
    "records": dim_product_final.count(),
    "status": "✅ Created (includes UNKNOWN product)"
})




In [0]:
display(spark.table(f"{GOLD_SCHEMA}.dim_product").limit(5))

### Przykład 4.3: dim_date

**Utworzona tabela dim_date:**

In [0]:
# Gold: dim_date (generowane z silver_orders jak w Lakeflow)
silver_orders = spark.table(f"{SILVER_SCHEMA}.orders_clean")

dim_date = (silver_orders
    .select("order_date")
    .filter(F.col("order_date").isNotNull())  # Filtruj NULL dates
    .distinct()
    .withColumn("date_key", F.date_format(F.col("order_date"), "yyyyMMdd").cast("int"))
    .withColumn("date", F.col("order_date"))
    .withColumn("year", F.year(F.col("order_date")))
    .withColumn("quarter", F.quarter(F.col("order_date")))
    .withColumn("month", F.month(F.col("order_date")))
    .withColumn("day", F.dayofmonth(F.col("order_date")))
    .withColumn("day_of_week", F.date_format(F.col("order_date"), "E"))
    .withColumn("is_weekend", 
        F.when(F.date_format(F.col("order_date"), "E").isin("Sat", "Sun"), 1).otherwise(0)
    )
    .select("date_key", "date", "year", "quarter", "month", "day", "day_of_week", "is_weekend")
    .orderBy("date_key")
)

dim_date.write.format("delta").mode("overwrite").saveAsTable(f"{GOLD_SCHEMA}.dim_date")

display({
    "table": f"{GOLD_SCHEMA}.dim_date",
    "records": dim_date.count(),
    "status": "✅ Created"
})

In [0]:
display(spark.table(f"{GOLD_SCHEMA}.dim_date").limit(10))

### Przykład 4.5: dim_store i dim_payment_method

In [0]:
# Gold: dim_store (generowane z silver_orders jak w Lakeflow)
dim_store = (silver_orders
    .select("store_id")
    .distinct()
    .withColumn("store_code", F.col("store_id"))
)

dim_store.write.format("delta").mode("overwrite").saveAsTable(f"{GOLD_SCHEMA}.dim_store")

# Gold: dim_payment_method (generowane z silver_orders jak w Lakeflow)
dim_payment_method = (silver_orders
    .select("payment_method_code")
    .distinct()
    .withColumn("payment_method_group",
        F.when(F.col("payment_method_code").isin("Credit Card", "Debit Card"), "Card")
        .when(F.col("payment_method_code") == "Cash", "Cash")
        .when(F.col("payment_method_code") == "PayPal", "Digital wallet")
        .otherwise("Other")
    )
)

dim_payment_method.write.format("delta").mode("overwrite").saveAsTable(f"{GOLD_SCHEMA}.dim_payment_method")

display({
    "dim_store": {"records": dim_store.count(), "status": "✅"},
    "dim_payment_method": {"records": dim_payment_method.count(), "status": "✅"}
})

### Przykład 4.6: fact_sales (tabela faktów)

In [0]:
# Gold: fact_sales (główna tabela faktów - zgodna z Lakeflow)
silver_orders = spark.table(f"{SILVER_SCHEMA}.orders_clean")

fact_sales = (silver_orders
    .select(
        # Klucze do wymiarów
        "order_id",
        "store_id",
        F.coalesce(F.col("customer_id"), F.lit("UNKNOWN")).alias("customer_id"),
        F.coalesce(F.col("product_id"), F.lit("UNKNOWN")).alias("product_id"),
        "payment_method_code",
        "order_date_key",
        "order_ts",
        
        # Miary
        "quantity",
        "unit_price",
        "discount_percent",
        "gross_amount",
        "discount_amount",
        "net_amount",
        
        # Flagi
        "is_return",
        "is_future_dated",
        "is_unknown_customer",
        "is_unknown_product",
        
        # Lineage
        "source_system"
    )
)

fact_sales.write.format("delta").mode("overwrite").saveAsTable(f"{GOLD_SCHEMA}.fact_sales")

display({
    "table": f"{GOLD_SCHEMA}.fact_sales",
    "records": fact_sales.count(),
    "status": "✅ Created"
})

### Podgląd fact_sales z joinami do wymiarów

In [0]:
# Query Star Schema - przykładowy raport sprzedaży
display(
    spark.sql(f"""
        SELECT 
            d.date,
            d.day_of_week,
            c.first_name || ' ' || c.last_name AS customer_name,
            c.customer_segment,
            p.product_name,
            p.brand,
            pm.payment_method_group,
            f.quantity,
            f.gross_amount,
            f.discount_amount,
            f.net_amount
        FROM {GOLD_SCHEMA}.fact_sales f
        JOIN {GOLD_SCHEMA}.dim_date d ON f.order_date_key = d.date_key
        JOIN {GOLD_SCHEMA}.dim_customer c ON f.customer_id = c.customer_id
        JOIN {GOLD_SCHEMA}.dim_product p ON f.product_id = p.product_id
        JOIN {GOLD_SCHEMA}.dim_payment_method pm ON f.payment_method_code = pm.payment_method_code
        WHERE f.is_return = 0 AND f.is_future_dated = 0
        ORDER BY f.net_amount DESC
        LIMIT 10
    """)
)

## Sekcja 5: Podsumowanie & Best Practices

### Co zostało osiągnięte?

✅ **1. Medallion Architecture Implementation (zgodna z Lakeflow)**

**🥉 Bronze Layer:**
- `bronze.customers_raw` - CSV z metadanymi (source_file_path, ingestion_ts, load_ts)
- `bronze.orders_raw` - JSON z source_system (batch/stream)
- `bronze.products_raw` - Parquet

**🥈 Silver Layer:**
- `silver.orders_clean` - z kalkulacjami (gross_amount, discount_amount, net_amount)
- `silver.orders_clean` - z flagami (is_return, is_future_dated, is_unknown_*)
- `silver.customers_type1` - SCD Type 1 (current state)
- `silver.customers_type2` - SCD Type 2 (__START_AT, __END_AT)

**🥇 Gold Layer (Star Schema):**
- `fact_sales` - tabela faktów z miarami i flagami
- `dim_customer` - snapshot z SCD2
- `dim_product` - z is_active, is_unknown
- `dim_date` - date_key, year, quarter, month, is_weekend
- `dim_store` - store_id, store_code
- `dim_payment_method` - payment_method_group (Card/Cash/Digital wallet)

✅ **2. SCD (Slowly Changing Dimensions)**
- **Type 1**: Overwrite (customers_type1)
- **Type 2**: Historical tracking z __START_AT/__END_AT (customers_type2, format Lakeflow)

✅ **3. Data Quality (zgodne z Lakeflow CONSTRAINTS)**
- valid_order_id: DROP ROW if NULL
- valid_customer: DROP ROW if NULL
- valid_product: DROP ROW if NULL
- valid_quantity: DROP ROW if NULL or 0
- valid_unit_price: DROP ROW if NULL or < 0

### Model Danych - Porównanie z Lakeflow

| Lakeflow (Streaming) | Notebook (Batch) | Status |
|---------------------|------------------|--------|
| `bronze_customers` (STREAMING TABLE) | `bronze.customers_raw` | ✅ |
| `bronze_orders` (STREAMING TABLE + FLOW) | `bronze.orders_raw` | ✅ |
| `bronze_products` (MATERIALIZED VIEW) | `bronze.products_raw` | ✅ |
| `silver_customers` (SCD2 AUTO CDC) | `silver.customers_type2` | ✅ |
| `silver_orders` (CONSTRAINTS) | `silver.orders_clean` | ✅ |
| `fact_sales` (STREAMING TABLE) | `gold.fact_sales` | ✅ |
| `dim_customer` (MV) | `gold.dim_customer` | ✅ |
| `dim_product` (MV) | `gold.dim_product` | ✅ |
| `dim_date` (MV) | `gold.dim_date` | ✅ |
| `dim_store` (MV) | `gold.dim_store` | ✅ |
| `dim_payment_method` (MV) | `gold.dim_payment_method` | ✅ |

### Kluczowe Wnioski

💡 **1. Lakeflow Streaming vs Batch**
```
Lakeflow: CREATE OR REFRESH STREAMING TABLE + FLOW
Notebook: spark.read → transform → write (batch)

Obie metody tworzą identyczny model danych!
```

💡 **2. Kalkulacje w Silver (nie Bronze)**
```python
# Silver orders - kalkulowane miary
gross_amount = quantity * unit_price
discount_amount = gross_amount * discount_percent / 100
net_amount = gross_amount - discount_amount
```

💡 **3. SCD Type 2 - format Lakeflow**
```python
# Lakeflow: AUTO CDC ... STORED AS SCD TYPE 2
# Notebook: Manual MERGE z __START_AT, __END_AT

# Query current: WHERE __END_AT IS NULL
# Query historical: WHERE date BETWEEN __START_AT AND __END_AT
```

💡 **4. Star Schema Query Pattern**
```sql
SELECT f.*, d.year, c.customer_segment, p.product_name
FROM fact_sales f
JOIN dim_date d ON f.order_date_key = d.date_key
JOIN dim_customer c ON f.customer_id = c.customer_id
JOIN dim_product p ON f.product_id = p.product_id
```

### Production Checklist

**Bronze Layer:**
- [x] Metadata columns (source_file_path, ingestion_ts, load_ts)
- [x] source_system tracking (batch/stream)
- [x] Multi-format support (CSV, JSON, Parquet)

**Silver Layer:**
- [x] Calculated measures (gross_amount, discount_amount, net_amount)
- [x] Quality flags (is_return, is_future_dated, is_unknown_*)
- [x] CONSTRAINT validation (DROP invalid rows)
- [x] SCD Type 2 for customers (__START_AT, __END_AT)

**Gold Layer:**
- [x] Star Schema (fact_sales + dimensions)
- [x] dim_customer (snapshot z SCD2)
- [x] dim_product (is_active, is_unknown)
- [x] dim_date (date_key, is_weekend)
- [x] dim_store, dim_payment_method

### Następne Kroki

**📚 Kolejne Notebooki:**
- **04_optimization_best_practices.ipynb** - Performance tuning
- **Warsztaty praktyczne** - End-to-end pipeline implementation

**🛠️ Lakeflow Pipeline:**
- Przejrzyj pliki SQL w `/Lakeflow/` dla production-ready streaming implementation

---

**Gratulacje!** 🎉 
Ukończyłeś implementację Medallion Architecture zgodną z modelem Lakeflow!
Jesteś gotowy do budowania production-grade data lakehouse pipelines!

## Sekcja 6: Czyszczenie Zasobów

**Uwaga:** Ta sekcja jest opcjonalna. Uruchom tylko jeśli chcesz usunąć wszystkie dane utworzone w tym notebooku.

### Opcja 1: Sprawdź utworzone zasoby (zalecane)

In [0]:
# Sprawdź utworzone zasoby (Star Schema)
medallion_tables = {
    "Bronze": [
        f"{BRONZE_SCHEMA}.customers_raw",
        f"{BRONZE_SCHEMA}.orders_raw",
        f"{BRONZE_SCHEMA}.products_raw"
    ],
    "Silver": [
        f"{SILVER_SCHEMA}.orders_clean",
        f"{SILVER_SCHEMA}.customers_type1",
        f"{SILVER_SCHEMA}.customers_type2"
    ],
    "Gold": [
        f"{GOLD_SCHEMA}.fact_sales",
        f"{GOLD_SCHEMA}.dim_customer",
        f"{GOLD_SCHEMA}.dim_product",
        f"{GOLD_SCHEMA}.dim_date",
        f"{GOLD_SCHEMA}.dim_store",
        f"{GOLD_SCHEMA}.dim_payment_method"
    ]
}

results = []
for layer, tables in medallion_tables.items():
    for table in tables:
        full_table = f"{CATALOG}.{table}"
        if spark.catalog.tableExists(full_table):
            count = spark.table(full_table).count()
            detail = spark.sql(f"DESCRIBE DETAIL {full_table}").collect()[0]
            size_mb = detail['sizeInBytes'] / (1024 * 1024)
            results.append({"layer": layer, "table": table, "records": count, "size_mb": round(size_mb, 2)})

display(spark.createDataFrame(results))

**Dane są zachowane dla dalszego użytku**

Aby usunąć wszystkie tabele, uruchom następną komórkę w sekcji opcjonalnej.

### Opcja 2: Usuń wszystkie zasoby (tylko jeśli naprawdę chcesz)

**UWAGA:** To usunie wszystkie tabele Silver i Gold utworzone w tym notebooku!

In [0]:
# Opcja 2: Usuń wszystkie zasoby (TYLKO JEŚLI JESTEŚ PEWIEN!)

# ⚠️  UWAGA: Odkomentuj poniższy kod tylko jeśli chcesz usunąć wszystko!

"""
print("=== 🗑️  USUWANIE ZASOBÓW MEDALLION ===\n")

# Lista tabel do usunięcia (Star Schema)
tables_to_drop = [
    # Silver
    f"{SILVER_SCHEMA}.orders_clean",
    f"{SILVER_SCHEMA}.customers_type1",
    f"{SILVER_SCHEMA}.customers_type2",
    # Gold - Star Schema
    f"{GOLD_SCHEMA}.fact_sales",
    f"{GOLD_SCHEMA}.dim_customer",
    f"{GOLD_SCHEMA}.dim_product",
    f"{GOLD_SCHEMA}.dim_date",
    f"{GOLD_SCHEMA}.dim_store",
    f"{GOLD_SCHEMA}.dim_payment_method"
]

print("Usuwanie tabel...\n")
for table in tables_to_drop:
    full_table = f"{CATALOG}.{table}"
    try:
        spark.sql(f"DROP TABLE IF EXISTS {full_table}")
        print(f"  ✓ Usunięto: {table}")
    except Exception as e:
        print(f"  ⚠️  Błąd przy {table}: {e}")

print("\n✅ Czyszczenie zakończone!")
"""

display({
    "status": "⚠️ KOD CZYSZCZENIA JEST ZAKOMENTOWANY",
    "info": "Odkomentuj powyższy kod tylko jeśli chcesz usunąć wszystkie zasoby",
    "zalecenie": "Zostaw dane dla kolejnych notebooków i warsztatów!",
    "next": "04_optimization_best_practices.ipynb"
})